
disord express server?

TODO: wire this to llm scaffold and virtual file system



In [ ]:

//const {doBrowserRequest} = importer.import('browser crawler tools')
//const {
//  safeurl, existingCache, storeCache, readCache,
//} = importer.import('domain cache tools')
const {request} = require('gaxios')
const url = require('url');
const {DEFAULT_API, SECRET, TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
const {request: requestDiscord} = importer.import('discord authorize')
const screenshotMonitor = importer.import('node screenshots')
const getToken = importer.import('discord express token endpoint')
const qs = require('qs');
const path = require('path')

async function getInstance(instanceId) {
  return await requestDiscord({
    method: 'GET',
    url: `applications/${DEFAULT_APPLICATION}/activity-instances/${instanceId}`
  })
}

let instances = {}
let express
let cors
try {
  express = require('express')
  cors = require('cors')
} catch (e) {
  console.error(e)
}

function replaceHome(data, authenticated) {
  data = data
      //.replace('https://github.com/briancullinan2', '/.proxy/github.com/briancullinan2')
      .replace(/<\/?iframe.*?>/gi, '')
  if(authenticated) {
    data = data.replace('<input type="radio" id="navigate" name="selected-tab" />', `
      <input type="radio" id="pizza" name="selected-tab" />
      <input type="radio" id="navigate" name="selected-tab" />
      `)
            .replace('<label for="navigate">Menu</label>', `
      <a class="nowButton" target="_blank" href="https://www.pizzahut.com">Order Pizza</a>
      <br />
      <br />
      <label for="navigate">Menu</label>`)
      
  }
  return data
}

async function handleRoute(req, res, next) {
  let authenticated = false
  //const cache = existingCache('https://www.briancullinan.com' + req.originalUrl, false)
  try {
    var url_parts = url.parse(req.url, true);
    var query = url_parts.query;
    //console.log(query)
    if(typeof query['launch_id'] != 'undefined' 
      && typeof query['frame_id'] != 'undefined' 
      && typeof query['user_id'] != 'undefined'
    ) {
      //let instance = await getInstance(query['instance_id'])
      //console.log(instance)
      if(typeof instances[query['launch_id']] == 'undefined') {
        instances[query['launch_id']] = []
      }
      instances[query['launch_id']].push([query['user_id'], query['frame_id']])
      console.log(instances)
      authenticated = true
    }

    // TODO: lookup user id from cookie?
    let proxySite = req.originalUrl.replace(/^\/|\/$/gi, '')
    if(!path.dirname(req.path.replace(/^\/|\/$/gi, '')).includes('.')
      || path.dirname(req.path.replace(/^\/|\/$/gi, '')).length == 1
    ) {
      proxySite = 'https://www.briancullinan.com' + req.originalUrl
    }
    if(!proxySite.includes('://')) {
      proxySite = 'https://' + proxySite
    }
    let proxyURL = url.parse(proxySite, true);

    let result = await request({
      url: proxySite,
      method: req.method,
      data: req.data,
    })
    //console.log(result)
    let data = result.data
    if(result.headers['content-type'].includes('text/html')) {
      data = data.replace('<head>', `<head>
<script type="text/javascript">
${importer.interpret('discord client code').code.replace('{TOKEN}', TOKEN).replace(/\{CLIENT_ID\}/gi, DEFAULT_APPLICATION)}
</script>
<base href="https://${DEFAULT_APPLICATION}.discordsays.com/.proxy/${proxyURL.host}/" target="_blank">
        `)

      // replace all external links with links to proxy service
      if(req.path == '/') {
        data = replaceHome(data, authenticated)
      }
      data = data.replace(/(href|src)="(.*?)"/gi, (match, p1, p2) => {
        if(p2.startsWith('/')) {
          return p1 + '="/.proxy/' + req.headers.host + p2 + '"'
        }
        if(p2.startsWith('https://')) {
          return p1 + '="/.proxy' + p2.replace('https://', '/') + '"'
        }
        return p1 + '="/.proxy/' + req.headers.host + '/' + path.dirname(req.path) + '/' + p2 + '"'
      })
      //data = data.replaceAll('/', `href="${proxyURL.host}/`)
      //data = data.replaceAll('href="https://', 'href="/.proxy/')
      //data = data.replaceAll('src="/', `src="/.proxy/${proxyURL.host}/`)
      //data = data.replaceAll('src="https://', 'src="/.proxy/')
    }
    res.setHeader('Content-Type', result.headers['content-type'])
    res.send(data)
  } catch(e) {
    console.error(e)
  }
  /*
  try {
    if(cache[])
    let response = await doBrowserRequest('https://www.briancullinan.com' + req.originalUrl, 
      readCache.bind(null, cache),
      storeCache.bind(null, cache),
      async (response, page) => {
        //if()
        //let html = await page.evaluate(() => document.querySelector('*').innerHTML)
        //console.log(response.headers())
      })
    res.setHeader('Content-Type', response.headers()['content-type'])
    return res.send(await response.buffer())
  } catch (e) {
    console.error(e)
  }
  */
  return next()
  //if(!req.get('Referer')) {
  //} else {
  //  console.log(req.get('Referer'))
  //}
}

async function getMonitor(req, res, next) {
  let index = parseInt(req.path.replace(/^\/|\/$/gi, '').split('/')[1])
  if(Number.isNaN(index)) {
    index = 0
  }
  let screenshot = await screenshotMonitor(index)
  if(!screenshot) {
    return next()
  }
  res.setHeader('Content-Type', 'image/png')
  return res.send(screenshot)
}

async function doClick(req, res, next) {
  try {
    const { Monitor } = require("node-screenshots");
    const MouseController = require('mouse-controller');
    const mc = new MouseController();
    let monitors = Monitor.all()
    mc.move(req.body.x * monitors[0].width * 3, req.body.y * monitors[0].height)
  } catch (e) {
    console.log(e)
  }
  return res.send('')
}

let app
if(express) {
  app = express();
  app.use(cors({
    origin: '*' 
  }))
  app.use(express.json());
  app.post('/click', doClick);
  app.get('/monitors/*', getMonitor);
  app.post('/api/token', getToken);
  app.get('*', handleRoute);
}

let server
//let activities = {}

async function discordExpress(activity) {
  if(!express) {
    return false
  }
  
  process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

  if(activity.participants.length == 0 && server) {
    server.close()
    server = null
  }

  // TODO: check if user was removed
  if(typeof instances[activity['instance_id']] == 'undefined') {
    instances[activity['instance_id']] = []
  }
  let removed = instances[activity['instance_id']].filter(i => !activity.participants.includes(i[0]))
  //if(removed) {
    // update our list because discord is the authority
    instances[activity['instance_id']] = instances[activity['instance_id']].filter(i => activity.participants.includes(i[0]))
  //}

  if(activity.length > 1 || server) {
    return
  }
  //if(typeof activities[activity.instance_id] == 'undefined') {
  //  activities[activity.instance_id] = []
  //}
  //activities[activity.instance_id] = participants

  return (server = await app.listen(3000, () => {
    console.log('Listening on: ', 3000)
  }))
}

function closeExpress(activity) {
  if(activity) {
    if(typeof instances[activity['instance_id']] != 'undefined') {
      instances[activity['instance_id']] = instances[activity['instance_id']].filter(i => activity.participants.includes(i[0]))
    }
    return
  }
  if(server) {
    server.close()
    server = null
  }
}

module.exports = {
  discordExpress,
  closeExpress,
  getInstance,
}




discord client code?


In [ ]:

let code
let userId

function sourceWindow() {
  return (window.parent.opener || window.parent)
}

document.addEventListener('click', async (evt) => {
  if(evt.target.tagName == 'A' && evt.target.getAttribute('href').includes('://')) {
    debugger


    sourceWindow().postMessage([1, {
      cmd: 'OPEN_EXTERNAL_LINK', 
      args: {url: evt.target.href}, 
      nonce: Math.random() * 1000,
      transfer: void 0
    }], '*')

    evt.preventDefault()
    return false
  }

  if(evt.target.tagName == 'A' && evt.target.getAttribute('href').includes('/.proxy/')) {
    const loggedInResponse = await fetch(evt.target.getAttribute('href'))
    const newContent = await loggedInResponse.text()
    //if(!newContent.includes('Cannot GET'))
    document.body.innerHTML = newContent.replace(/<\/?html>/gi, '')
                                        .replace(/<script.*?>[\s\S]*?<\/script>/gi, '');
  }
  return true
})

let result = {}
let params = (window.location.search.split('?')[1] || '').split('&')
for(var param in params) {
  if (params.hasOwnProperty(param)) {
    let paramParts = params[param].split('=')
    result[paramParts[0]] = decodeURIComponent(paramParts[1] || "")
  }
}

document.addEventListener("DOMContentLoaded", (event) => {

  debugger

  if(typeof result.frame_id == 'undefined') {
    return
  }

})



discord client auth code?


In [ ]:

let expiry
let result
let initNonce = Math.round(Math.random() * 1000)

window.addEventListener('message', async (evt) => {

  document.querySelector('h1').innerHTML = '<pre>' + JSON.stringify(evt.data, null, 2) + '</pre>'

  const sourceWindow = () => {
    return (window.parent.opener || window.parent)
  }

  if(evt.data[0] == 3 && typeof evt.data[1].frame_id != 'undefined') {

    result = evt.data[1] // get it from page init instead of query params
    // send handshake
    sourceWindow().postMessage([0, {
      v: 1,
      encoding: 'json',
      client_id: '{CLIENT_ID}',
      frame_id: evt.data[1].frame_id,
      nonce: initNonce,
      transfer: void 0
    }], 'https://discord.com' || '*')
  }

  if(evt.data[0] == 1 && evt.data[1].cmd == 'DISPATCH') {

    // start authorization chain
    sourceWindow().postMessage([1, {
      cmd: 'AUTHORIZE',
      args: {
        client_id: '{CLIENT_ID}',
        prompt: 'none',
        response_type: 'code',
        scope: ['applications.commands', 'identify', 'guilds.members.read', 'rpc.activities.write'],
        state: '',
      },
      nonce: Math.random() * 1000,
      transfer: void 0
    }], 'https://discord.com' || '*')

  }

  if(evt.data[1].cmd == 'AUTHORIZE') {
    code = evt.data[1].data.code

    const response = await fetch("{BASE_URI}api/token", {
      method: "POST",
      headers: {
        "Content-Type": "application/json",
      },
      credentials: 'same-origin',
      body: JSON.stringify({
        code,
        launch_id: result.launch_id,
        frame_id: result.frame_id,
      }),
    });
    const { access_token, expires_in, session_id } = await response.json();
    result.session_id = session_id
    
    debugger

    // TODO: take authenticate commands
  
    sourceWindow().postMessage([1, {
      cmd: 'AUTHENTICATE',
      args: {
        access_token: access_token
      },
      nonce: Math.random() * 1000,
      transfer: void 0
    }], 'https://discord.com' || '*')

    expiry = Date.now() + expires_in
  }

  if(evt.data[1].cmd == 'AUTHENTICATE') {
    // TODO: some authenticated page update
    if(result.session_id)
      window.location = '{BASE_URI}?t=' + Date.now() + '&session=' + result.session_id
    /*
    userId = evt.data[1].data.user.id
    window.launch_id = result.launch_id
    window.frame_id = result.frame_id
    window.user_id = userId

    if(typeof result.frame_id != 'undefined' && typeof result.user_id == 'undefined') {
      let absolute = '{BASE_URI}register?user_id=' + userId 
        + '&frame_id=' + result.frame_id
        + '&launch_id=' + result.launch_id
        + '&expiry=' + expiry + '&t=' + Date.now()
      // cache busting for authenticated users
      const loggedInResponse = await fetch(absolute, {
        method: 'POST',
        credentials: 'include'
      })
      //const newContent = await loggedInResponse.text()
      //document.body.innerHTML = newContent.replace(/<\/?html>/gi, '')
      //                                    .replace(/<script.*?>[\s\S]*?<\/script>/gi, '');
    }
    */
  }
})




start activity server?


In [ ]:
const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {discordExpress, closeExpress, getInstance} = importer.import('discord express server')

const ALL_COMMANDS = [
  'launch',
]

// bot commands using new API, same names as above but lower-case
async function activityCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = discordExpress
  interactionsCommands['endActivity'] = closeExpress

  cmdDef = {
    'name': 'launch',
    'description': 'Launch an activity',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('launch')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'launch')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = activityCommands




check discord commands?


In [ ]:

const {getCommands} = importer.import('discord api')

module.exports = getCommands



start a bunch of discord services?


In [ ]:
const {spawn, spawnSync} = require("child_process");
const path = require('path')

const DISCORD_SERVICES = [
  void 0,
  void 0, // TODO: Orbb bot
  'discord mid journey commands',
  'sync deceptive chat',
  'sync discord llm tools',
  'discord handy tools',
  'discord remote control',
  'discord music player',
]

const APPLICATION_IDS = [
  void 0,
  void 0,
  '1335464182111535124',
  '1335472379232780401',
  '1335479586858733568',
  '1335483072602177626',
  '1335491680630997034',
  '1335769252409380884',
]

function launchDiscord() {
  for(let i = 0; i < DISCORD_SERVICES.length; i++) {
    if(!DISCORD_SERVICES[i]) {
      continue
    }
    spawn('node', ['--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', DISCORD_SERVICES[i]], {
      env: Object.assign({}, process.env, {
        TOKENPATH: '/Users/briancullinan/.credentials/discord-bot' + (i ? i : '') + '.txt',
        DEFAULT_APPLICATION: APPLICATION_IDS[i]
      }),
      stdio: [0, 1, 2],
      //detached: true,
      //shell: true,
      cwd: path.dirname(__dirname)
    })
  
  }
}

module.exports = launchDiscord



discord authenticate instances?


In [ ]:
const {INSTANCES, SESSIONS} = importer.import('discord gateway')

/*
async function registerInstance(req, res, next) {
  var url_parts = url.parse(req.url, true);
  var query = url_parts.query;

  if(typeof query['launch_id'] != 'undefined' 
    && typeof query['frame_id'] != 'undefined' 
    && typeof query['user_id'] != 'undefined'
  ) {
    //let instance = await getInstance(query['instance_id'])
    //console.log(instance)
    if(typeof INSTANCES[query['launch_id']] == 'undefined') {
      INSTANCES[query['launch_id']] = []
    }
    INSTANCES[query['launch_id']].push([query['user_id'], query['frame_id']])
    console.log(INSTANCES)
    //console.log(req.cookies.discord_bot_expiry)
    let expiry = new Date(parseInt(query['expiry']))

    //authenticated = true
    // TODO: cookies
    res.cookie('discord_bot_session', query['user_id'], { 
      expires: expiry
    })
    res.cookie('discord_bot_frame', query['frame_id'], { 
      expires: expiry
    })
  }

  return res.send('')
}
*/

const ALLOWED_USERS = [
  '1019970191603544164'
]

async function authenticateRoute(req, res, next) {
  var url_parts = url.parse(req.url, true);
  var query = url_parts.query;
  //console.log(query, SESSIONS, query.session && typeof SESSIONS[query.session] != 'undefined' 
  //  && ALLOWED_USERS.includes(SESSIONS[query.session]))
  if(query.session && typeof SESSIONS[query.session] != 'undefined' 
    && ALLOWED_USERS.includes(SESSIONS[query.session])) {
    req.authenticated = query.session
    return next()
  }

  // TODO: check if the user is in fact apart of the activity
  req.authenticated = false

  return next()
}


module.exports = {
  //registerInstance,
  authenticateRoute,
  INSTANCES,
  SESSIONS,
  
}



discord express token endpoint?


In [ ]:
const {INSTANCES, SESSIONS} = importer.import('discord gateway')
const {request} = require('gaxios')
const qs = require('qs');
const {DEFAULT_API, SECRET, TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
var crypto = require('crypto')

async function getToken(req, res, next) {
  //console.log(req.body)
  let result = await request({
    url: DEFAULT_API + 'oauth2/token',
    method: 'POST',
    headers: {
      'Content-Type': 'application/x-www-form-urlencoded',
      //'Authorization': 'Bot ' + TOKEN
    },
    data: qs.stringify({
      client_id: DEFAULT_APPLICATION,
      client_secret: SECRET,
      grant_type: 'authorization_code',
			code: req.body.code,
		})
  })

  let expiry = new Date(Date.now() + result.data.expires_in)

  if(typeof req.body['launch_id'] != 'undefined' 
    && typeof req.body['frame_id'] != 'undefined')
  {
    let userResult = await request({
      method: 'GET',
      url: DEFAULT_API + `oauth2/@me`,
      headers: {
        'Authorization': 'Bearer ' + result.data.access_token
      }
    })

    // TODO: generate nonce for client to use on future commands to authenticate with our server quickly instead of relying on cookies

    if(typeof INSTANCES[req.body['launch_id']] == 'undefined') {
      INSTANCES[req.body['launch_id']] = []
    }
    INSTANCES[req.body['launch_id']].push([userResult.data.user.id, req.body['frame_id']])
    console.log(INSTANCES)

    result.data.session_id = crypto.randomBytes(16).toString('hex')
    SESSIONS[result.data.session_id] = userResult.data.user.id

    res.cookie('discord_bot_session', userResult.data.user.id, {
      path: '/.proxy/',
      expires: expiry
    })
    res.cookie('discord_bot_frame', req.body['frame_id'], { 
      path: '/.proxy/',
      expires: expiry
    })
  }

  res.cookie('discord_bot_expiry', Date.now() + result.data.expires_in, { 
    path: '/.proxy/',
    expires: expiry
  })
  res.setHeader('Content-Type', 'application/json')
  return res.send(JSON.stringify(result.data))
}

module.exports = getToken
